In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 再現性確保

import os
import random
import numpy as np
import tensorflow as tf

os.environ['PYTHONHASHSEED'] = '42'
os.environ['TF_DETERMINISTIC_OPS'] = '1'

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
# データクリーニング

'''

%cd /content/drive/MyDrive/assignment-main(new)/Trainee/time-series-prediction

df = pd.read_csv('/content/drive/MyDrive/assignment-main(new)/Trainee/time-series-prediction/stock_price.csv')

def parse_volume(s):
  if 'B' in s:
    return float(s.replace('B', '')) * 1e9
  elif 'M' in s:
    return float(s.replace('M', '')) * 1e6
  else:
    return float(s)

def clean(df):
  df_new = df.copy()
  df_new = df_new[::-1]
  df_new = df_new.rename(columns={'日付け': 'date', '終値': 'close', '始値': 'open', '高値': 'high', '安値': 'low', '出来高': 'volume', '変化率 %': 'return'})
  df_new['date'] = pd.to_datetime(df_new['date'])
  df_new = df_new.set_index('date')
  df_new['volume'] = df_new['volume'].apply(parse_volume)
  df_new['return'] = df_new['return'].str.replace('%', '').astype(float) / 100
  return df_new

cleaned_df = clean(df)


from google.colab import files
cleaned_df.to_csv('my_cleaned_stock_price.csv')
files.download('my_cleaned_stock_price.csv')

'''

"\n\n%cd /content/drive/MyDrive/assignment-main(new)/Trainee/time-series-prediction\n\ndf = pd.read_csv('/content/drive/MyDrive/assignment-main(new)/Trainee/time-series-prediction/stock_price.csv')\n\ndef parse_volume(s):\n  if 'B' in s:\n    return float(s.replace('B', '')) * 1e9\n  elif 'M' in s:\n    return float(s.replace('M', '')) * 1e6\n  else:\n    return float(s)\n\ndef clean(df):\n  df_new = df.copy()\n  df_new = df_new[::-1]\n  df_new = df_new.rename(columns={'日付け': 'date', '終値': 'close', '始値': 'open', '高値': 'high', '安値': 'low', '出来高': 'volume', '変化率 %': 'return'})\n  df_new['date'] = pd.to_datetime(df_new['date'])\n  df_new = df_new.set_index('date')\n  df_new['volume'] = df_new['volume'].apply(parse_volume)\n  df_new['return'] = df_new['return'].str.replace('%', '').astype(float) / 100\n  return df_new\n\ncleaned_df = clean(df)\n\n\nfrom google.colab import files\ncleaned_df.to_csv('my_cleaned_stock_price.csv')\nfiles.download('my_cleaned_stock_price.csv')\n\n"

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/assignment-main(new)/Trainee/time-series-prediction/my_cleaned_stock_price.csv')
df

,date,close,open,high,low,volume,return
0,1987-02-12,182.4,182.4,182.4,182.4,1.280000e+09,0.1625
1,1987-02-13,172.5,176.5,176.5,172.5,4.223800e+08,-0.0543
2,1987-02-16,175.5,166.7,175.5,165.7,3.605100e+08,0.0174
3,1987-02-17,182.4,176.5,184.3,175.5,8.815100e+08,0.0393
4,1987-02-18,191.2,186.3,191.2,186.3,7.950900e+08,0.0482
...,...,...,...,...,...,...,...
9197,2024-07-26,157.9,159.3,159.6,157.9,1.550800e+08,-0.0013
9198,2024-07-29,159.7,158.7,160.2,158.4,1.262800e+08,0.0114
9199,2024-07-30,158.7,158.8,159.2,158.0,1.381400e+08,-0.0063
9200,2024-07-31,160.4,158.2,160.7,158.1,1.739100e+08,0.0107


In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df

,close,open,high,low,volume,return
date,,,,,,
1987-02-12,182.4,182.4,182.4,182.4,1.280000e+09,0.1625
1987-02-13,172.5,176.5,176.5,172.5,4.223800e+08,-0.0543
1987-02-16,175.5,166.7,175.5,165.7,3.605100e+08,0.0174
1987-02-17,182.4,176.5,184.3,175.5,8.815100e+08,0.0393
1987-02-18,191.2,186.3,191.2,186.3,7.950900e+08,0.0482
...,...,...,...,...,...,...
2024-07-26,157.9,159.3,159.6,157.9,1.550800e+08,-0.0013
2024-07-29,159.7,158.7,160.2,158.4,1.262800e+08,0.0114
2024-07-30,158.7,158.8,159.2,158.0,1.381400e+08,-0.0063


In [ ]:
import holidays

# 終値からRSIを計算
def compute_rsi(data, window=14):
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def engineer(df_old):

  df_new = df_old.copy()

  df_new['sin_dayofyear'] = np.sin(2 * np.pi * df_new.index.dayofyear / 365)
  df_new['cos_dayofyear'] = np.cos(2 * np.pi * df_new.index.dayofyear / 365)
  df_new['sin_dayofmonth'] = np.sin(2 * np.pi * df_new.index.day / 30)
  df_new['cos_dayofmonth'] = np.cos(2 * np.pi * df_new.index.day / 30)
  df_new['sin_dayofweek'] = np.sin(2 * np.pi * df_new.index.dayofweek / 7)
  df_new['cos_dayofweek'] = np.sin(2 * np.pi * df_new.index.dayofweek / 7)

  # 日本の休日
  jp_holidays = holidays.Japan()
  df_new['holiday'] = df_new.index.map(lambda x: 1 if x in jp_holidays else 0)

  # 移動平均、相対力指数
  sma_list = [5, 10, 20, 60]
  for i in sma_list:
    df_new[f'sma_{i}'] = df_new['close'].rolling(window=i).mean()
    df_new[f'ema_{i}'] = df_new['close'].ewm(span=i, adjust=False).mean()
    df_new[f'rsi_{i}'] = compute_rsi(df_new, window=i)

  # 標準偏差
  for i in sma_list:
    df_new[f'std_{i}'] = df_new['close'].rolling(window=i).std()

  # 急激な上昇をとらえる
  for i in range(len(sma_list)):
    for j in range(i+1, len(sma_list)):
      df_new[f'sma_{sma_list[i]}vs{sma_list[j]}'] = df_new[f'sma_{sma_list[i]}'] / df_new[f'sma_{sma_list[j]}']
      # ゴールデンクロス
      df_new[f'gc_{sma_list[i]}vs{sma_list[j]}'] = (df_new[f'sma_{sma_list[i]}'].shift(1) < df_new[f'sma_{sma_list[j]}'].shift(1)) & (df_new[f'sma_{sma_list[i]}'] >= df_new[f'sma_{sma_list[j]}'])
      # デッドクロス
      df_new[f'dc_{sma_list[i]}vs{sma_list[j]}'] = (df_new[f'sma_{sma_list[i]}'].shift(1) > df_new[f'sma_{sma_list[j]}'].shift(1)) & (df_new[f'sma_{sma_list[i]}'] <= df_new[f'sma_{sma_list[j]}'])
      # MACD
      df_new[f'macd_{sma_list[i]}vs{sma_list[j]}'] = df_new[f'ema_{sma_list[i]}'] - df_new[f'ema_{sma_list[j]}']
      df_new[f'macd_signal_{sma_list[i]}vs{sma_list[j]}'] = df_new[f'macd_{sma_list[i]}vs{sma_list[j]}'].ewm(span=9, adjust=False).mean()

  # ADX
  epsilon = 1e-6
  df_new['+DM'] = df_new['high'] - df_new['high'].shift(1)
  df_new['-DM'] = df_new['low'].shift(1) - df_new['low']
  df_new['TR'] = np.maximum.reduce([
    df_new['high'] - df_new['low'],
    abs(df_new['high'] - df_new['close'].shift(1)),
    abs(df_new['low'] - df_new['close'].shift(1))
    ])
  df_new['+DI'] = df_new['+DM'].rolling(window=14).sum() / (df_new['TR'].rolling(window=14).sum() + epsilon) * 100
  df_new['-DI'] = df_new['-DM'].rolling(window=14).sum() / (df_new['TR'].rolling(window=14).sum() + epsilon) * 100
  df_new['DX'] = abs((df_new['+DI'] - df_new['-DI']) / (df_new['+DI'] + df_new['-DI'] + epsilon)) * 100
  df_new['ADX'] = df_new['DX'].rolling(window=14).mean()

  # ストキャスティクス
  df_new['%K'] = (df_new['close'] - df_new['low'].rolling(window=14).min()) / (df_new['high'].rolling(window=14).max() - df_new['low'].rolling(window=14).min()) * 100
  df_new['%D'] = df_new['%K'].rolling(window=3).mean()


  #df_new = df_new.drop(['open', 'high', 'low'], axis=1)

  return df_new

engineered_df = engineer(df)
engineered_df

,close,open,high,low,volume,return,sin_dayofyear,cos_dayofyear,sin_dayofmonth,cos_dayofmonth,...,macd_signal_20vs60,+DM,-DM,TR,+DI,-DI,DX,ADX,%K,%D
date,,,,,,,,,,,,,,,,,,,,,
1987-02-12,182.4,182.4,182.4,182.4,1.280000e+09,0.1625,0.674444,0.738326,5.877853e-01,-0.809017,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1987-02-13,172.5,176.5,176.5,172.5,4.223800e+08,-0.0543,0.687053,0.726608,4.067366e-01,-0.913545,...,-0.123653,-5.9,9.9,9.9,NaN,NaN,NaN,NaN,NaN,NaN
1987-02-16,175.5,166.7,175.5,165.7,3.605100e+08,0.0174,0.723644,0.690173,-2.079117e-01,-0.978148,...,-0.292928,-1.0,6.8,9.8,NaN,NaN,NaN,NaN,NaN,NaN
1987-02-17,182.4,176.5,184.3,175.5,8.815100e+08,0.0393,0.735417,0.677615,-4.067366e-01,-0.913545,...,-0.403124,8.8,-9.8,8.8,NaN,NaN,NaN,NaN,NaN,NaN
1987-02-18,191.2,186.3,191.2,186.3,7.950900e+08,0.0482,0.746972,0.664855,-5.877853e-01,-0.809017,...,-0.358767,6.9,-10.8,8.8,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-26,157.9,159.3,159.6,157.9,1.550800e+08,-0.0013,-0.425000,-0.905193,-7.431448e-01,0.669131,...,-1.887455,0.6,-1.1,1.7,5.830904,-6.413994,2.100004e+03,8.685501e+08,56.818182,59.848485
2024-07-29,159.7,158.7,160.2,158.4,1.262800e+08,0.0114,-0.471160,-0.882048,-2.079117e-01,0.978148,...,-1.673703,0.6,-0.5,2.3,14.583333,-16.369047,1.733334e+03,4.649221e+08,75.609756,63.839616
2024-07-30,158.7,158.8,159.2,158.0,1.381400e+08,-0.0063,-0.486273,-0.873807,-1.133108e-15,1.000000,...,-1.482857,-1.0,0.4,1.7,6.918239,-14.150943,2.913044e+02,4.649221e+08,50.000000,60.809313


In [ ]:
df_new = engineered_df['2020-01-01':]
train = df_new[:'2022-12-31']
embargo_train_valid = df_new['2023-01-01':'2023-03-31']
valid = df_new['2023-04-01':'2023-08-31']
embargo_valid_test = df_new['2024-01-01':'2024-06-30']
test = df_new['2024-07-01':]

In [ ]:
input_seq_len = min(60, len(embargo_valid_test))
output_seq_len = len(test)

In [ ]:
def create_dataset(df, input_seq_len=60, output_seq_len=22):
  data = df.values
  X, y = [], []
  if len(df) - input_seq_len - output_seq_len < 0:
    raise ValueError('input_seq_len or output_seq_len is too large')
  for i in range(len(df) - input_seq_len - output_seq_len):
    X.append(data[i:i+input_seq_len])
    y.append(data[i+input_seq_len:i+input_seq_len+output_seq_len, 0])
  return np.array(X), np.array(y)

X_train, y_train = create_dataset(train, input_seq_len=input_seq_len, output_seq_len=output_seq_len)
X_valid, y_valid = create_dataset(valid, input_seq_len=input_seq_len, output_seq_len=output_seq_len)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

X_scaler = StandardScaler()
X_train_scaled = X_scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_valid_scaled = X_scaler.transform(X_valid.reshape(-1, X_valid.shape[-1])).reshape(X_valid.shape)

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).reshape(y_train.shape)
y_valid_scaled = y_scaler.transform(y_valid.reshape(-1, 1)).reshape(y_valid.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras import regularizers, layers, models, Input
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow.keras.backend as K

In [ ]:
# LSTM


dropout_rate = 0.2

lstm_model = Sequential()
lstm_model.add(LSTM(512, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), return_sequences=True, kernel_initializer=GlorotUniform(seed=42)))
lstm_model.add(Dropout(dropout_rate))
lstm_model.add(LSTM(256, return_sequences=True))
lstm_model.add(Dropout(dropout_rate))
lstm_model.add(LSTM(128, return_sequences=True))
lstm_model.add(Dropout(dropout_rate))
lstm_model.add(LSTM(64, return_sequences=True))
lstm_model.add(Dropout(dropout_rate))
lstm_model.add(LSTM(32, return_sequences=False))
lstm_model.add(Dropout(dropout_rate))
lstm_model.add(Dense(output_seq_len))

lstm_model.summary()

lr_schedule = CosineDecay(initial_learning_rate=0.001, decay_steps = (len(X_train_scaled) // 64) * 1000)
optimizer = Adam(learning_rate=lr_schedule)
lstm_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
es = EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='min', restore_best_weights=True)
lstm_model.fit(X_train_scaled, y_train_scaled, epochs=1000, batch_size=64, validation_data=(X_valid_scaled, y_valid_scaled), shuffle=False, callbacks=[es])

plt.figure(figsize=(12, 8))
plt.plot(lstm_model.history.history['loss'], label='Train Loss')
plt.plot(lstm_model.history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 60, 512)             │       1,189,888 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 60, 512)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 60, 256)             │         787,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 60, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 60, 128)             │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 60, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 60, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 60, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 23)                  │             759 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,237,047 (8.53 MB)

 Trainable params: 2,237,047 (8.53 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 0.8414 - mae: 0.7981 - val_loss: 2.0452 - val_mae: 1.3880
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - loss: 0.6590 - mae: 0.6886 - val_loss: 1.6129 - val_mae: 1.2209
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 0.5157 - mae: 0.6051 - val_loss: 1.3142 - val_mae: 1.1068
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - loss: 0.4466 - mae: 0.5591 - val_loss: 1.0438 - val_mae: 0.9881
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - loss: 0.3778 - mae: 0.5146 - val_loss: 0.8602 - val_mae: 0.9037
Epoch 6/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - loss: 0.3775 - mae: 0.5143 - val_loss: 0.6808 - val_mae: 0.8043
Epoch 7/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - loss: 0.4248 - mae: 0.5358 - val_loss: 0.5795 - val_mae: 0.7434
Epoch 8/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - loss: 0.3412 - mae: 0.4873 - val_loss: 0.5982 - val_mae: 0.7615
Epoch 9/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - lo

KeyboardInterrupt: 

In [ ]:
# LSTM + Attention


class Attention(Layer):
  def __init_(self, **kwargs):
    super(Attention, self).__init__(**kwargs)

  def build(self, input_shape):
    self.W = self.add_weight(shape=(input_shape[-1], 1),
                             initializer='glorot_uniform',
                             trainable=True)
    self.b = self.add_weight(shape=(input_shape[1], 1),
                             initializer='zeros',
                             trainable=True)
    super(Attention, self).build(input_shape)

  def call(self, x):
    e = K.tanh(K.dot(x, self.W) + self.b)
    a = K.softmax(e, axis=1)
    output = x*a
    return K.sum(output, axis=1)

def create_attention_lstm(input_shape, output_dim):

  inputs = Input(shape=input_shape)

  x = LSTM(256, return_sequences=True, kernel_regularizer=regularizers.l2(0.001))(inputs)
  x = Dropout(0.2)(x)
  x = LSTM(128, return_sequences=True, kernel_regularizer=regularizers.l2(0.001))(x)
  x = Dropout(0.2)(x)
  x = LSTM(64, return_sequences=True, kernel_regularizer=regularizers.l2(0.001))(x)
  x = Dropout(0.2)(x)

  x = Attention()(x)

  x = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
  x = Dropout(0.2)(x)
  x = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
  x = Dropout(0.2)(x)

  outputs = Dense(output_dim)(x)
  lstm_attention_model = Model(inputs, outputs)
  return lstm_attention_model

lstm_attention_model = create_attention_lstm(input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), output_dim=output_seq_len)
lstm_attention_model.summary()
lstm_attention_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

es = EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='min', restore_best_weights=True)

history = lstm_attention_model.fit(
    X_train_scaled, y_train_scaled,
    epochs=1000,
    batch_size=64,
    validation_data=(X_valid_scaled, y_valid_scaled),
    shuffle=False,
    callbacks=[es]
    )

plt.figure(figsize=(12, 8))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Transformer

class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, embed_dim):
      super().__init__()
      self.embed_dim = embed_dim

  def call(self, x):
      # x.shape[1] はシーケンスの長さ、x.shape[-1] は埋め込み次元数
      pos = tf.cast(tf.range(start=0, limit=tf.shape(x)[1], delta=1), dtype=tf.float32)  # posをfloat32にキャスト
      i = tf.cast(tf.range(start=0, limit=self.embed_dim, delta=1), dtype=tf.float32)  # iもfloat32にキャスト

      # 位置エンコーディングの計算
      angle_rates = 1 / tf.pow(10000., (2 * (i // 2)) / self.embed_dim)
      angles = tf.expand_dims(pos, axis=-1) * angle_rates  # 位置エンコーディング
      cos = tf.math.sin(angles[:, 0::2])  # 奇数番目の次元
      sin = tf.math.cos(angles[:, 1::2])  # 偶数番目の次元
      pos_encoding = tf.concat([sin, cos], axis=-1)

      # 位置エンコーディングを元の入力に加算
      return x + tf.expand_dims(pos_encoding, axis=0)

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1):
  # Multi-Head Attention
  x = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(inputs, inputs)
  x = Dropout(dropout)(x)
  x = LayerNormalization(epsilon=1e-6)(x + inputs)

  # Feed-Forward部分
  x_ff = Dense(ff_dim, activation='gelu')(x)
  x_ff = Dropout(dropout)(x_ff)
  x_ff = Dense(inputs.shape[-1])(x_ff)
  x = LayerNormalization(epsilon=1e-6)(x + x_ff)
  return x

def build_transformer_model(input_shape, output_dim):
  inputs = Input(shape=input_shape)

  # Position Encoding
  embed_dim = 128
  x = Dense(embed_dim)(inputs)
  x = Dropout(0.2)(x)
  x = PositionalEncoding(embed_dim)(x)

  for _ in range(2):
    x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=128, dropout=0.2)

  x = GlobalAveragePooling1D()(x)

  x = Dense(64, activation='gelu')(x)
  x = Dropout(0.2)(x)

  outputs = Dense(output_dim)(x)

  model = models.Model(inputs, outputs)
  return model

transformer_model = build_transformer_model(
    input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]),
    output_dim=output_seq_len
)
transformer_model.summary()

transformer_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

es = EarlyStopping(monitor='val_loss', patience=200, verbose=1, mode='min', restore_best_weights=True)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=50, verbose=1, min_lr=1e-5)

history = transformer_model.fit(
    X_train_scaled, y_train_scaled,
    epochs=1000,
    batch_size=64,
    validation_data=(X_valid_scaled, y_valid_scaled),
    shuffle=False,
    callbacks=[es, lr_schedule]
)

plt.figure(figsize=(12, 8))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

In [ ]:
X_test = embargo_valid_test[-input_seq_len:].values
X_test_scaled = X_scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [ ]:
def pred(model, X_test_scaled, y_scaler):
  y_test_pred_scaled = model.predict(X_test_scaled.reshape(1, X_test_scaled.shape[0], X_test_scaled.shape[1]))
  y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled).flatten()
  return y_test_pred

lstm_test_pred = pred(lstm_model, X_test_scaled, y_scaler)
lstm_attention_test_pred = pred(lstm_attention_model, X_test_scaled, y_scaler)
transformer_test_pred = pred(transformer_model, X_test_scaled, y_scaler)

ensemble_test_pred = lstm_test_pred * 0.0 + lstm_attention_test_pred * 0.5 + transformer_test_pred * 0.5

In [ ]:
'''
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

best_rmse = float('inf')
best_mae = float('inf')
best_weights = (0, 0, 0)

# 重みの候補（例：0.0, 0.1, ..., 1.0）
weight_candidates = np.arange(0, 1.1, 0.1)

for i in weight_candidates:
    for j in weight_candidates:
        for k in weight_candidates:
            # 合計が1.0の組み合わせのみ考慮
            if abs(i + j + k - 1.0) > 1e-6:
                continue

            ensemble_test_pred = (
                lstm_test_pred * i +
                lstm_attention_test_pred * j +
                transformer_test_pred * k
            )

            rmse = np.sqrt(mean_squared_error(test['close'], ensemble_test_pred))
            mae = mean_absolute_error(test['close'], ensemble_test_pred)

            if rmse < best_rmse and mae < best_mae:
                best_rmse = rmse
                best_mae = mae
                best_weights = (i, j, k)

print('✅ Best RMSE:', best_rmse)
print('✅ Best MAE:', best_mae)
print('✅ Best Weights (LSTM, LSTM+Attention, Transformer):', best_weights)
'''

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_model(model, y_test_pred):
    # モデルがNoneの場合は名前を'Ensemble'に設定
    model_name = model.name if model else 'Ensemble'

    rmse = np.sqrt(mean_squared_error(test['close'], y_test_pred))
    mae = mean_absolute_error(test['close'], y_test_pred)

    print(f'{model_name} RMSE: {rmse}')
    print(f'{model_name} MAE: {mae}')

# モデルごとに評価
evaluate_model(lstm_model, lstm_test_pred)
evaluate_model(lstm_attention_model, lstm_attention_test_pred)
evaluate_model(transformer_model, transformer_test_pred)

# アンサンブルモデルを評価
evaluate_model(None, ensemble_test_pred)


In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(test.index, lstm_test_pred, label='LSTM')
plt.plot(test.index, lstm_attention_test_pred, label='LSTM + Attention')
plt.plot(test.index, transformer_test_pred, label='Transformer')
plt.plot(test.index, ensemble_test_pred, label='Ensemble')
plt.plot(df_new.index, df_new['close'], label='Actual')
plt.title('Actual vs Predicted')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(test.index, lstm_test_pred, label='LSTM')
plt.plot(test.index, lstm_attention_test_pred, label='LSTM + Attention')
plt.plot(test.index, transformer_test_pred, label='Transformer')
plt.plot(test.index, ensemble_test_pred, label='Ensemble')
plt.plot(test.index, test['close'], label='Actual')
plt.title('Actual vs Predicted')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()